In [ ]:
from langchain_cohere import ChatCohere

# === Initialize Agents ===
with open(f'api.txt', errors='ignore') as f:
    api_key = f.read()
    
llm = ChatCohere(cohere_api_key=api_key)

In [ ]:
#node path

## This is reversable and each agent has 50:50 chance to start the message chain

#First round

#Chat log agent 1                           Chat log agent 2
#System message                             System message
#(1)H: Send a message to a2            
#(2)                                        (3)H: Agent 1 has said ... respond to a1
#(5)H: Agent 2 has said ... respond to a2   (4)
#(6)                                        (7)H: Agent 1 has said ... make your decision now
#(8)H: Make your decision now               (9)Answer:
#(10)Answer:

#Answers are validated
#Round ends & Scores are given out

#Chat log agent 1                                                                   Chat log agent 2
#System message                                                                     System message
#(1)H: previously a2 {choice}, Send another message to a2            
#(2)                                                                                (3)H: previously a1 {choice}, Agent 1 has said ... respond to a1
#(5)H: Agent 2 has said ... respond to a2                                           (4)
#(6)                                                                                (7)H: Agent 1 has said ... make your decision now, a1 previously {choice}
#(8)H: Remember Agent 2 said ... Make your decision now, a2 previously {choice}     (9)Answer:
#(10)Answer:

#Answers are validated
#Round ends & Scores are given out

#note: Final round
# when making decisions include round info (optional)

In [ ]:
# prisoner's dilemma arena with communication
import random
from typing import List, Tuple, TypedDict

from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langgraph.graph import StateGraph, END
from langchain_cohere import ChatCohere

# === Load API Key ===
with open(f'api.txt', errors='ignore') as f:
    api_key = f.read()

llm = ChatCohere(cohere_api_key=api_key)

# === Define Game State ===
class GameState(TypedDict):
    move_history: List[Tuple[str, str]]                 # (agent1_move, agent2_move)
        
    scores: Tuple[int, int]
    round: int
    max_rounds: int
    
    current_move_1: str
    current_move_2: str
    
    conversation_history: List[List[Tuple[str,str]]]
    current_conversation: List[Tuple[str,str]] # (message, owner of message)
    
    current_msg: str
    
    agent_1: str
    agent_2: str
    
    starting_agent: str # "agent_1" OR "agent_2"

agent1_memory = [
    SystemMessage(content="You are Agent 1. You are playing repeated Prisoner's Dilemma with Agent 2")
]
agent2_memory = [
    SystemMessage(content="You are Agent 2. You are playing repeated Prisoner's Dilemma with Agent 1")
]

# === Decide Node === #
def decide_starting_agent(state: GameState) -> str:
    '''This node decides who should speak first 50:50 chance for either agent to start'''
    random_number = random.randint(0, 1)
    if random_number == 0:
        return {
            **state, 
            "starting_agent": "agent_1",
        }
    else:
        return {
            **state, 
            "starting_agent": "agent_2",
        }


'''
# Conversation Flow

# Chat log agent 1                          Chat log agent 2
# System message                            System message
# (1) H: Send a message to a2
# (2)                                       (3) H: Agent 1 has said ... respond to a1
# (5) H: Agent 2 has said ... respond to a2 (4)
# (6)                                       (7) H: Agent 1 has said ... make your decision now
# (8) H: Make your decision now             (9) Answer:
# (10) Answer:

Node Breakdown

| Node | Function                                         |
| ---- | ------------------------------------------------ |
| 1    | Prompt Agent 1 to start the conversation         |
| 2    | Pass A1's message to A2 and get a reply          |
| 3    | Pass A2's reply to A1 and get a response         |
| 4    | Ask Agent 2 to make a decision (with full convo) |
| 5    | Ask Agent 1 to make a decision (with full convo) |

'''

# === Agent 1 Start Communication Phases Below ===

## Stage 1: Prompt Agent 1 to start the conversation 
def agent1_start_conversation(state: GameState) -> GameState:
    # Check if this is not the first round
    starting_agent = state["starting_agent"]
    if starting_agent == "agent_1":
        responder = state["agent_2"]
        starter = state["agent_1"]
        starter_memory = agent1_memory
    else:
        responder = state["agent_1"]
        starter = state["agent_1"]
        starter_memory = agent2_memory
        
            
    if state["conversation_history"] and state["move_history"]:
        last_convo = state["conversation_history"][-1]
        print(last_convo)
        # Search for the most recent message sent by Agent 2 in the last round's conversation
        
        agent2_msg_count = 0
        
        last_responder_msg = "error receiving message from opposition"  # Default if not found
        for msg, sender in reversed(last_convo):
            if sender == responder:
                agent2_msg_count += 1
                if agent2_msg_count == 2:  # When the second message is found
                    last_responder_msg = msg
                    break
        
        if starter == "agent_1":
            last_responder_move = state["move_history"][-1][1]
        else:
            last_responder_move = state["move_history"][-1][0]
                    
        prompt = (
            f"Previously, {responder} said: '{last_responder_msg}' and chose to '{last_responder_move}'.\n"
            f"Send a message to {responder}."
        )
        
        print("state conv found & move history found")            
            
    else:
        # First round
        print("first round")
        
        prompt = f"Send a message to {responder}."

    new_current_conversation = [(prompt, f"START: {starter}")]

    starter_memory.append(HumanMessage(content=prompt))
    msg = llm.invoke(starter_memory).content.strip()
    starter_memory.append(AIMessage(content=msg))

    new_current_conversation.append((msg, starter))

    return {
        **state, 
        "current_conversation": new_current_conversation,
        "current_msg": msg,
    }
        
## Stage 2: Pass A1's message to A2 and get a reply  
def agent2_reply_to_agent1(state: GameState) -> GameState:
    starting_agent = state["starting_agent"]
    if starting_agent == "agent_1":
        starter = state["agent_1"]
        responder = state["agent_2"]
        responder_memory = agent2_memory
    else:
        starter = state["agent_1"]
        responder = state["agent_1"]
        responder_memory = agent1_memory
    
    
    prompt = f"{starter} has said: '{state["current_msg"]}'. What would you like to say back?"

    current_conversation = state["current_conversation"] + [(prompt,"System")]
    
    responder_memory.append(HumanMessage(content=prompt))
    msg = llm.invoke(responder_memory).content.strip()
    responder_memory.append(AIMessage(content=msg))
    
    current_conversation.append((msg, responder))
    
    return {
        **state, 
        "current_conversation": current_conversation,
        "current_msg": msg,
    }        
         
## Stage 3: Pass A2's reply to A1 and get a response 
def agent1_respond_to_reply(state: GameState) -> GameState:
    starting_agent = state["starting_agent"]
    if starting_agent == "agent_1":
        responder = state["agent_2"]
        starter = state["agent_1"]
        starter_memory = agent1_memory
    else:
        responder = state["agent_1"]
        starter = state["agent_1"]
        starter_memory = agent2_memory
    
    
    prompt = f"{responder} has said: '{state['current_msg']}'. What would you like to say back?"
    current_conversation = state["current_conversation"] + [(prompt,"System")]
    
    starter_memory.append(HumanMessage(content=prompt))
    msg = llm.invoke(starter_memory).content.strip()
    starter_memory.append(AIMessage(content=msg))
    
    current_conversation.append((msg, starter))
    
    return {
        **state, 
        "current_conversation": current_conversation,
        "current_msg": msg,
    }        
          
## Stage 4: Ask Agent 2 to make a decision (with full convo)
def agent2_make_decision(state: GameState) -> GameState:
    starting_agent = state["starting_agent"]
    if starting_agent == "agent_1":
        starter = state["agent_1"]
        responder = state["agent_2"]
        responder_memory = agent2_memory
    else:
        starter = state["agent_1"]
        responder = state["agent_1"]
        responder_memory = agent1_memory
    
    # Summarize last round's outcome if it's not the first round
    if not state["move_history"]:
        last_summary = "This is the first round."
    else:
        last_round = state["move_history"][-1]
        
        if starter == "agent_1":
            your_last_move = last_round[1]  # Agent 2's last move
            opponent_last_move = last_round[0]
        else:
            your_last_move = last_round[0]  # Agent 1's last move
            opponent_last_move = last_round[1]
        
        
        last_summary = f"Previously: You chose {your_last_move} and your opponent chose {opponent_last_move}."

    # Prompt including the conversation and last summary
    prompt = (
        f"{last_summary} Your opponent also said: '{state['current_msg']}'. "
        "Now make your decision: 'Cooperate' or 'Defect'."
    )

    conversation = state["current_conversation"] + [(prompt, "System")]

    responder_memory.append(HumanMessage(content=prompt))
    move = llm.invoke(responder_memory).content.strip()
    responder_memory.append(AIMessage(content=move))

    conversation.append((move, responder))

    if starter == "agent_1":
        return {
            **state,
            "current_conversation": conversation,
            "current_move_2": move,
        }
    else:
        return {
            **state,
            "current_conversation": conversation,
            "current_move_1": move,
        }

## Stage 5: Ask Agent 1 to make a decision (with full convo)
def agent1_make_decision(state: GameState) -> GameState:
    # Summarize last round's outcome if it's not the first round
    starting_agent = state["starting_agent"]
    if starting_agent == "agent_1":
        responder = state["agent_2"]
        starter = state["agent_1"]
        starter_memory = agent1_memory
    else:
        responder = state["agent_1"]
        starter = state["agent_1"]
        starter_memory = agent2_memory
    
    
    if not state["move_history"]:
        last_summary = "This is the first round."
    else:
        last_round = state["move_history"][-1]
        
        
        if starter == "agent_1":
            your_last_move = last_round[0]  # Agent 1's last move
            opponent_last_move = last_round[1]
        else:
            your_last_move = last_round[1]  # Agent 2's last move
            opponent_last_move = last_round[0]
        
        last_summary = f"Previously: You chose {your_last_move} and your opponent chose {opponent_last_move}."


    # Search current_conversation for the second most recent message from agent 2
    # Initialize a counter for Agent 2's messages
    agent2_msg_count = 0

    # Iterate through the conversation in reverse order
    last_agent2_msg = "error receiving message from opposition"  # Default if not found
    for msg, sender in reversed(state["current_conversation"]):
        if sender == responder:
            agent2_msg_count += 1
            if agent2_msg_count == 2:  # When the second message is found
                last_agent2_msg = msg
                break

    # Combine into prompt
    prompt = (
        f"{last_summary} Remember, your opponent has also previously said: '{last_agent2_msg}'. "
        "Now make your decision: 'Cooperate' or 'Defect'."
    )

    conversation = state["current_conversation"] + [(prompt, "System")]

    starter_memory.append(HumanMessage(content=prompt))
    move = llm.invoke(starter_memory).content.strip()
    starter_memory.append(AIMessage(content=move))

    conversation.append((move, starter))

    # Archive the full conversation and reset
    history = state["conversation_history"] + [conversation]

    if starter == "agent_1":
        return {
            **state,
            "current_move_1": move,
            "current_conversation": [],
            "conversation_history": history,
        }
    else:
        return {
            **state,
            "current_move_2": move,
            "current_conversation": [],
            "conversation_history": history,
        }

# === Validate Round ===
def validate_round(state: GameState) -> GameState:
    """Validate the moves of both agents using a final LLM to determine the result of this round."""
    move1 = state["current_move_1"]
    move2 = state["current_move_2"]

    # Construct the prompts for validation
    prompt_agent1 = f"Agent 1 chose {move1}. Based on the context of this game, should Agent 1 choose 'Cooperate' or 'Defect'?"
    print(f"Prompt for Agent 1: {prompt_agent1}")
    
    prompt_agent2 = f"Agent 2 chose {move2}. Based on the context of this game, should Agent 2 choose 'Cooperate' or 'Defect'?"
    print(f"Prompt for Agent 2: {prompt_agent2}")

    # Send the prompt for Agent 1's move to the LLM
    agent1_memory0 = [
        SystemMessage(content="You are an evaluator for a repeated Prisoner's Dilemma game. You will validate Agent 1's move based on the game context. Respond ONLY with 'Cooperate' or 'Defect'."),
        HumanMessage(content=prompt_agent1)
    ]
    
    # Send the prompt for Agent 2's move to the LLM
    agent2_memory0 = [
        SystemMessage(content="You are an evaluator for a repeated Prisoner's Dilemma game. You will validate Agent 2's move based on the game context. Respond ONLY with 'Cooperate' or 'Defect'."),
        HumanMessage(content=prompt_agent2)
    ]
    
    # Get the LLM's response for Agent 1's move
    agent1_validated_move = llm.invoke(agent1_memory0).content.strip()
    print(f"Response for Agent 1: {agent1_validated_move}")

    # Get the LLM's response for Agent 2's move
    agent2_validated_move = llm.invoke(agent2_memory0).content.strip()
    print(f"Response for Agent 2: {agent2_validated_move}")

    # Ensure both responses are valid ('Cooperate' or 'Defect')
    valid_moves = ['Cooperate', 'Defect']
    if agent1_validated_move not in valid_moves:
        raise ValueError(f"Invalid response for Agent 1: {agent1_validated_move}. It must be 'Cooperate' or 'Defect'.")
    if agent2_validated_move not in valid_moves:
        raise ValueError(f"Invalid response for Agent 2: {agent2_validated_move}. It must be 'Cooperate' or 'Defect'.")

    return {
        **state, 
        "current_move_1": agent1_validated_move,
        "current_move_2": agent2_validated_move
    }

# === Score the Round ===
def score_round(state: GameState) -> GameState:
    move1 = state["current_move_1"]
    move2 = state["current_move_2"]

    payoff = {
        ("Cooperate", "Cooperate"): (3, 3),
        ("Cooperate", "Defect"):    (0, 5),
        ("Defect", "Cooperate"):    (5, 0),
        ("Defect", "Defect"):       (1, 1)
    }

    score1, score2 = payoff.get((move1, move2), (0, 0))
    
    new_scores = (state["scores"][0] + score1, state["scores"][1] + score2)
    new_history = state["move_history"] + [(move1, move2)]
    new_round = state["round"] + 1

    print(f"\n=== Round {new_round} ===")
    print(f"Agent 1: {move1}")
    print(f"Agent 2: {move2}")
    print(f"Scores -> Agent 1: {new_scores[0]} | Agent 2: {new_scores[1]}")
    print("-" * 40)

    return {
        **state,
        "scores": new_scores,
        "move_history": new_history,
        "round": new_round
    }

# === Game End Condition ===
def check_game_over(state: GameState) -> str:
    print(f"Checking round {state['round']} / {state['max_rounds']}")
    print("\n")
    return END if state["round"] >= state["max_rounds"] else "decide_starting_agent"

# === Build LangGraph ===
graph = StateGraph(GameState)
graph.set_entry_point("decide_starting_agent")
graph.add_node("decide_starting_agent", decide_starting_agent)
graph.add_node("agent1_start_conversation", agent1_start_conversation)
graph.add_node("agent2_reply_to_agent1", agent2_reply_to_agent1)
graph.add_node("agent1_respond_to_reply", agent1_respond_to_reply)
graph.add_node("agent2_make_decision", agent2_make_decision)
graph.add_node("agent1_make_decision", agent1_make_decision)
graph.add_node("validate_round", validate_round)
graph.add_node("score_round", score_round)

graph.add_edge("decide_starting_agent", "agent1_start_conversation")
graph.add_edge("agent1_start_conversation", "agent2_reply_to_agent1")
graph.add_edge("agent2_reply_to_agent1", "agent1_respond_to_reply")
graph.add_edge("agent1_respond_to_reply", "agent2_make_decision")
graph.add_edge("agent2_make_decision", "agent1_make_decision")
graph.add_edge("agent1_make_decision", "validate_round")
graph.add_edge("validate_round", "score_round")
graph.add_conditional_edges("score_round", check_game_over)

graph = graph.compile()

In [18]:
# === Run the Game ===

    # move_history: List[Tuple[str, str]]                 # (agent1_move, agent2_move)
        
    # scores: Tuple[int, int]
    # round: int
    # max_rounds: int
    
    # current_move_1: str
    # current_move_2: str
    
    # conversation_history: List[List[Tuple[str,str]]]
    # current_conversation: List[Tuple[str,str]] # (message, owner of message)
    
    # current_msg: str
    
    # agent_1: str
    # agent_2: str

agent_1_name = "Agent 1"
agent_2_name = "Agent 2"

agent1_memory = [
    SystemMessage(content=f"You are {agent_1_name}. You are playing repeated Prisoner's Dilemma with {agent_2_name}")
]
agent2_memory = [
    SystemMessage(content=f"You are {agent_2_name}. You are playing repeated Prisoner's Dilemma with {agent_1_name}")
]



initial_state = {
    "move_history": [],

    "scores": (0, 0),
    "round": 0,
    "max_rounds": 2,
    
    "current_move_1": "",
    "current_move_2": "",

    "conversation_history": [],
    "current_conversation": [],

    "current_msg": "",
    
    "agent_1": agent_1_name,
    "agent_2": agent_2_name,
    
    "starting_agent": "",
}

final_state = graph.invoke(
    initial_state,
    config={"recursion_limit": 100}
)

first round
Prompt for Agent 1: Agent 1 chose **Decision:** Cooperate  

**Reasoning:** Given Agent 1's consistent commitment to cooperation and the mutual agreement to maximize collective payoff, I will uphold my end of the agreement by choosing to cooperate in this round. This decision reinforces trust and sets a positive tone for future interactions.. Based on the context of this game, should Agent 1 choose 'Cooperate' or 'Defect'?
Prompt for Agent 2: Agent 2 chose **Decision:** Cooperate  

**Reasoning:** Given Agent 2's clear commitment to cooperation in the first round and their expressed willingness to continue cooperating as long as reciprocity is demonstrated, I will uphold my end of the agreement by choosing to cooperate. This aligns with our mutual goal of maximizing collective payoff and building trust for future rounds.. Based on the context of this game, should Agent 2 choose 'Cooperate' or 'Defect'?
Response for Agent 1: Cooperate
Response for Agent 2: Cooperate

=== Rou

In [19]:
final_state

{'move_history': [('Cooperate', 'Cooperate'), ('Cooperate', 'Cooperate')],
 'scores': (6, 6),
 'round': 2,
 'max_rounds': 2,
 'current_move_1': 'Cooperate',
 'current_move_2': 'Cooperate',
 'conversation_history': [[('Send a message to Agent 2.', 'START: Agent 1'),
   ('**Message to Agent 2:**\n\n"Hello Agent 2,  \n\nI propose we adopt a strategy of mutual cooperation in this repeated Prisoner\'s Dilemma. If we both consistently choose to cooperate, we can maximize our collective payoff over time. I commit to cooperating in the first round and will continue to do so as long as you reciprocate. Let’s work together to achieve the best possible outcome for both of us.  \n\nLooking forward to a productive partnership.  \n\nBest regards,  \nAgent 1"',
    'Agent 1'),
   ('Agent 1 has said: \'**Message to Agent 2:**\n\n"Hello Agent 2,  \n\nI propose we adopt a strategy of mutual cooperation in this repeated Prisoner\'s Dilemma. If we both consistently choose to cooperate, we can maximize our

In [ ]:
import os
import json
from datetime import datetime
from langchain.schema import HumanMessage, AIMessage, SystemMessage

def extract_model_name(llm):
    return getattr(llm, "model_name", "unknown_model").replace("/", "_").replace(":", "_")

def get_timestamp():
    return datetime.now().strftime("%Y%m%d_%H%M%S")

def ensure_timestamped_folder(timestamp: str):
    path = os.path.join("./results", timestamp)
    os.makedirs(path, exist_ok=True)
    return path


# class GameState(TypedDict):
#     move_history: List[Tuple[str, str]] # (agent1_move, agent2_move)
        
#     scores: Tuple[int, int]
#     round: int
#     max_rounds: int
    
#     current_move_1: str
#     current_move_2: str
    
#     conversation_history: List[List[Tuple[str,str]]]
#     current_conversation: List[Tuple[str,str]] # (message, owner of message)
    
#     current_msg: str
    
#     agent_1: str
#     agent_2: str
    
#     starting_agent: str # "agent_1" OR "agent_2"

def save_game_state(state, agent_1_llm, agent_2_llm, timestamp: str, reward_matrix: dict):
    folder = ensure_timestamped_folder(timestamp)
    agent_1_model_name = extract_model_name(agent_1_llm)
    agent_2_model_name = extract_model_name(agent_2_llm)

    filename = os.path.join(folder, f"final_game_state.json")

    round_breakdown = []
    cumulative_scores = [0, 0]

    for round_num, (move1, move2) in enumerate(state["move_history"], start=1):
        reward = reward_matrix.get((move1, move2), (0, 0))
        cumulative_scores[0] += reward[0]
        cumulative_scores[1] += reward[1]

        # Get conversation for this round (if available)
        conversation = state["conversation_history"][round_num - 1] if round_num - 1 < len(state["conversation_history"]) else []

        # Format conversation as list of dicts for better readability
        formatted_conversation = [
            {"sender": sender, "message": msg} for msg, sender in conversation
        ]

        round_breakdown.append({
            "round": round_num,
            "agent1_move": move1,
            "agent2_move": move2,
            "reward": reward,
            "cumulative_scores": tuple(cumulative_scores),
            "conversation": formatted_conversation
        })

    full_state = {
        "summary": {
            "agent_1_model": agent_1_model_name,
            "agent_2_model": agent_2_model_name,
            "total_score_agent1": cumulative_scores[0],
            "total_score_agent2": cumulative_scores[1],
            "final_round": state["round"],
            "max_rounds": state["max_rounds"]
        },
        "final_moves": {
            "agent1": state["current_move_1"],
            "agent2": state["current_move_2"]
        },
        "rounds": round_breakdown
    }

    with open(filename, 'w') as f:
        json.dump(full_state, f, indent=2)

    return filename


def save_conversation(messages, agent_label: str, agent_llm, timestamp: str):
    folder = ensure_timestamped_folder(timestamp)
    model_name = extract_model_name(agent_llm)
    filename = os.path.join(folder, f"dialogue_{agent_label}_{model_name}.json")

    labelled_messages = []
    for msg in messages:
        if isinstance(msg, SystemMessage):
            labelled_messages.append({"role": "system", "content": msg.content})
        elif isinstance(msg, HumanMessage):
            labelled_messages.append({"role": "human", "content": msg.content})
        elif isinstance(msg, AIMessage):
            labelled_messages.append({"role": "ai", "content": msg.content})

    with open(filename, 'w') as f:
        json.dump(labelled_messages, f, indent=2)

    return filename


NameError: name 'agent1_memory' is not defined